# Big Five Personalities

blablabla

# Previos

## Importación de Librerías

In [ ]:
from sklearn.cluster import KMeans 
from sklearn import metrics
from sklearn import cluster, datasets
from sklearn.metrics import silhouette_samples, silhouette_score

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.patches as mpatches

## Parámetros generales

In [ ]:
#Parámetros para mejorar la visualización de los dataframes
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 50)

# Parámetros para gráficos
plt.style.use('seaborn')

plt.rcParams.update({'font.size': 25})      
plt.rc('axes', labelsize=20)
plt.rc('legend', fontsize=18)  
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 
plt.rcParams['axes.titlesize'] = 25
plt.rcParams["figure.figsize"] = (15,6)

# Big Five

In [ ]:
df = pd.read_csv('data\data_final.csv', sep = '\t')

Eliminamos datos con NAN

In [ ]:
df = df.dropna()

Eliminamos columnas que no se utilizarán, por ahora

In [ ]:
try:
    del df['dateload']
    del df['screenw']
    del df['screenh']
    del df['introelapse']
    del df['testelapse']
    del df['endelapse']
    del df['IPC']
    del df['country']
    del df['lat_appx_lots_of_err']
    del df['long_appx_lots_of_err']
except:
    pass

Filtramos los datos que tengan algún cero en sus respuestas

In [ ]:
for i in range(0,50):
    col = df.columns[i]
    df = df.loc[df[col]!=0]

Pasamos el tiempo de respuesta a segundos

In [ ]:
df.loc[:, 'EXT1_E': 'OPN10_E'] = df.loc[:, 'EXT1_E': 'OPN10_E']/1000

# EDA

## Primer Area de Preguntas

### Respuestas a preguntas

In [ ]:
EXT_count = df.loc[:, 'EXT1':'EXT10'].apply(pd.value_counts)
print('Tabla de conteo de respuestas para cada pregunta')
display(EXT_count)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(EXT_count.apply(lambda x: x/x.sum(), axis = 0), annot = True, cmap = 'Blues',
           fmt = '.1g', linewidths = .01)
plt.title('Heatmap escalado por cantidad de respuestas a cada pregunta')
plt.show()

### Tiempos de respuestas

In [ ]:
print('Tiempos promedios en respuestas de las preguntas')
df.loc[:, 'EXT1_E':'EXT10_E'].mean(axis = 0)

In [ ]:
# Tiempos de respuestas promedios por respuesta a cada pregunta
tiempos_ext = pd.DataFrame(index = range(1,6))
for i in range(1,11):
    tiempos_ext[f'EXT{i}_T'] = df[[f'EXT{i}', f'EXT{i}_E']].groupby(f'EXT{i}').mean()[f'EXT{i}_E']

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(tiempos_ext, annot = True, cmap = 'Blues',
           fmt = '.3g', linewidths = .01)
plt.title('Tiempo promedio gastado en responder cada pregunta')
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(tiempos_ext.loc[:, 'EXT2_T':'EXT10_T'], annot = True, cmap = 'Blues',
           fmt = '.3g', linewidths = .01)
plt.title('Tiempo promedio gastado en responder cada pregunta')
plt.show()

# Trabajo con las respuestas

In [ ]:
cat_quest = {
            'EXT1': 1, 'EXT2': -1, 'EXT3': 1, 'EXT4': -1, 'EXT5': 1,
            'EXT6': -1, 'EXT7': 1, 'EXT8': -1, 'EXT9': 1, 'EXT10': -1,
            'EST1':-1, 'EST2':1, 'EST3':-1, 'EST4':1, 'EST5':-1,
            'EST6':-1, 'EST7':-1, 'EST8':-1, 'EST9':-1, 'EST10':-1,
            'AGR1':-1, 'AGR2':1, 'AGR3':-1, 'AGR4':1, 'AGR5':-1,
            'AGR6':1, 'AGR7':-1, 'AGR8':1, 'AGR9':1, 'AGR10':1,
            'CSN1':1, 'CSN2':-1, 'CSN3':1, 'CSN4':-1, 'CSN5':1,
            'CSN6':-1, 'CSN7':1, 'CSN8':-1, 'CSN9':-1, 'CSN10':1,
            'OPN1':1, 'OPN2':-1, 'OPN3':1, 'OPN4':-1, 'OPN5':1,
            'OPN6':-1, 'OPN7':1, 'OPN8':1, 'OPN9':1, 'OPN10':1
        }

Normalización de las respuestas a -1,0,1 e inversión en la respuesta

In [ ]:
for col in cat_quest.keys():
    df[col] = (df[col]-3)/2 # Normalización a -1.0.1
    df[col] = df[col]*cat_quest[col] # Inversión de la escala

Eliminación respuestas con tiempos negativos o menores a 0.5

In [ ]:
# Se seleccionan las preguntas de 1 a 10 de cada sección
areas_names = ['EXT', 'EST', 'AGR', 'CSN', 'OPN']
col_times = [area + str(i)+'_E' for area in areas_names for i in range(1,11)]

In [ ]:
time_col_quantiles = {}
for col in col_times:
    time_col_quantiles[col] = df[col].quantile(0.99)

In [ ]:
for col in time_col_quantiles.keys():
    df = df.loc[(df[col] <= time_col_quantiles[col]) & # filtrar valores menores a este quantile
               (df[col] >= 0.5)] # filtrar valores menores a 0.5 segundos y negativos

# PCA

In [ ]:
df.shape

In [ ]:
df_quest = df.loc[:, 'EXT1':'OPN10'].copy()

In [ ]:
df_quest.head()

## Función Auxiliar

In [ ]:
def predic_kmeans_plot(dataset, ncluster):
    
    kmeans = KMeans(n_clusters = ncluster, random_state=1)
    kmeans.fit(dataset)
    cluster_pred = kmeans.fit_predict(dataset)
    
    pca = PCA(n_components=dataset.shape[1])
    pca_result = pca.fit_transform(dataset)
    suma = np.cumsum(pca.explained_variance_ratio_)
    pca_data = pca.transform(dataset)
    
    per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
    labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
    
    pca_df = pd.DataFrame(pca_data, columns=labels)
    
    pca_df['Cluster'] = cluster_pred
    for i in range(0,ncluster):
        pca_df.loc[pca_df.Cluster == i,'Cluster'] = 'Cluster {}'.format(i)

    fig, ax = plt.subplots()
    
    sns.scatterplot(pca_df.PC1, pca_df.PC2, hue = pca_df['Cluster'])
    
    plt.title('ACP aplicado a todos los datos',fontsize=30)
    plt.xlabel('PC1 - Var. Exp.: {0}%'.format(per_var[0]),fontsize=25)
    plt.ylabel('PC2 - Var. Exp.: {0}%'.format(per_var[1]),fontsize=25)
    plt.show()
    return pca_df['Cluster']

In [ ]:
def PCA_apply(df):
    standard_datos = StandardScaler().fit(df)
    datos_norm = standard_datos.transform(df)
    pca = PCA(n_components=12)
    pca_result = pca.fit_transform(datos_norm)
    suma = np.cumsum(pca.explained_variance_ratio_)
    pca_data = pca.transform(datos_norm)
    
    plt.rcParams['figure.figsize'] = (15,8)

    per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
    labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
    
    fig, ax = plt.subplots()
    pca_df = pd.DataFrame(pca_data, index=df.index, columns=labels)
    ax.scatter(pca_df.PC1, pca_df.PC2)
    plt.title('ACP aplicado a todos los datos',fontsize=30)
    plt.xlabel('PC1 - Var. Exp.: {0}%'.format(per_var[0]),fontsize=25)
    plt.ylabel('PC2 - Var. Exp.: {0}%'.format(per_var[1]),fontsize=25)

    plt.show()
    
    fig, ax = plt.subplots()
    df_pca_componentes = pd.DataFrame(pca.components_, columns = df.columns).T
    
    X = list(df_pca_componentes[0])
    Y = list(df_pca_componentes[1])
    nombres = df_pca_componentes.index
    
    for i, type in enumerate(nombres):
        x = X[i]
        y = Y[i]
        plt.scatter(x, y , marker = 'o')
        plt.text(x,y+0.01,type, fontsize = 17)
    
    plt.title('Contribución de Variables a Primeras 2 Componentes',fontsize=30)
    plt.xlabel('PC1 - Var. Exp.: {0}%'.format(per_var[0]),fontsize=25)
    plt.ylabel('PC2 - Var. Exp.: {0}%'.format(per_var[1]),fontsize=25)

    plt.axhline(y = 0, alpha = 1)
    plt.axvline(x = 0, alpha = 1)
    plt.rcParams['figure.figsize'] = (15,8)

    plt.tight_layout()
    plt.show()
    return pca

In [ ]:
#pca_quest = PCA_apply(df_quest)

## Kmeans a respuestas

In [ ]:
cluster_kmeans = predic_kmeans_plot(df_quest, 5)

## Kmeans a tiempos de respuestas

In [ ]:
col_times_2_10 = [area + str(i)+'_E' for area in areas_names for i in range(2,11)]
cluster_kmeans_tiempos = predic_kmeans_plot(df[col_times_2_10], 5)